In [1]:
#|default_exp carmen_backend

In [2]:
#hide
!pip install -Uqq --upgrade --force-reinstall langchain langchain-openai langchain-anthropic
!pip install -Uqq langchain_experimental

In [3]:
#|export
import os
import json

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

from retrying import retry

from langchain_core.tools import tool
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

models = {
    "openai": {
        "generation": "gpt-3.5-turbo",
        "agent": "gpt-3.5-turbo"
    },
    "anthropic": {
        "generation": "claude-3-sonnet-20240229",
        "agent": "claude-3-opus-20240229"
    }
}

model_provider = os.getenv("MODEL_PROVIDER", "openai")

GAME_STATE_FILE = "./carmen_game_state.json"
MAX_HOPS = 5

In [4]:
#|export
def get_llm(provider, purpose):
    model = models[provider][purpose]
    
    if model_provider == "openai":
        return ChatOpenAI(api_key=OPENAI_API_KEY, model=model)
    elif model_provider == "anthropic":
        return ChatAnthropic(api_key=ANTHROPIC_API_KEY, model=model)
    else:
        raise Exception(f"Unknown model provider: {model_provider}")
        
def get_generation_llm():
    return get_llm(model_provider, "generation")

def get_agent_llm():
    return get_llm(model_provider, "agent")

In [5]:
#|export
def get_game_states():
    if not os.path.exists(GAME_STATE_FILE):
        return {}
    
    with open(GAME_STATE_FILE, "r") as f:
        return json.load(f)

In [6]:
#|export
def store_game_state(game_state: dict):
    game_states = get_game_states()
    game_states[game_state["case_id"]] = game_state

    with open(GAME_STATE_FILE, "w") as f:
        json.dump(game_states, f)

In [7]:
#|export
def get_game_state(case_id: str):
    return get_game_states()[case_id]

In [8]:
get_game_states()

{'a3f3c1d6-8a4b-4f18-bd3e-2a7b0d5d2e0e': {'case_id': 'a3f3c1d6-8a4b-4f18-bd3e-2a7b0d5d2e0e',
  'suspect_name': 'Baron von Schnitzel',
  'current_city': 'Chennai',
  'stolen_item': "Big Ben's bell",
  'hops': ['London',
   'Paris',
   'New York City',
   'Tokyo',
   'Sydney',
   'Rio de Janeiro'],
  'next_hop': 3},
 '3e12b9a1-1f8b-4d3b-aa9b-5d2d19d6c8d1': {'case_id': '3e12b9a1-1f8b-4d3b-aa9b-5d2d19d6c8d1',
  'suspect_name': 'Dastardly Dan',
  'current_city': 'Sydney',
  'stolen_item': 'Samurai Sword',
  'hops': ['Tokyo', 'Sydney', 'Dublin', 'Berlin', 'Cairo', 'New York City'],
  'next_hop': 1}}

In [9]:
store_game_state(
    {
        "case_id": 1,
        "b": 2,
    }
)

In [10]:
!cat {GAME_STATE_FILE}

{"a3f3c1d6-8a4b-4f18-bd3e-2a7b0d5d2e0e": {"case_id": "a3f3c1d6-8a4b-4f18-bd3e-2a7b0d5d2e0e", "suspect_name": "Baron von Schnitzel", "current_city": "Chennai", "stolen_item": "Big Ben's bell", "hops": ["London", "Paris", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}, "3e12b9a1-1f8b-4d3b-aa9b-5d2d19d6c8d1": {"case_id": "3e12b9a1-1f8b-4d3b-aa9b-5d2d19d6c8d1", "suspect_name": "Dastardly Dan", "current_city": "Sydney", "stolen_item": "Samurai Sword", "hops": ["Tokyo", "Sydney", "Dublin", "Berlin", "Cairo", "New York City"], "next_hop": 1}, "1": {"case_id": 1, "b": 2}}

In [11]:
!rm {GAME_STATE_FILE}

In [12]:
#|export
famous_cities = [
    "Paris", "New York City", "London", "Tokyo", "Rome",
    "Sydney", "Hong Kong", "Venice", "Barcelona", "Rio de Janeiro",
    "Cape Town", "Dubai", "San Francisco", "Moscow", "Amsterdam",
    "Prague", "Buenos Aires", "Mumbai", "Istanbul", "Bangkok",
    "Singapore", "Dublin", "Florence", "Berlin", "Kyoto",
    "Vienna", "Athens", "Seoul", "Beijing", "Los Angeles",
    "Mexico City", "Toronto", "Vancouver", "Zurich", "Madrid",
    "Edinburgh", "Las Vegas", "Cairo", "San Diego", "Shanghai",
    "Washington D.C.", "Stockholm", "Dubrovnik", "Jerusalem", "Budapest",
    "Oslo", "Lisbon", "Montreal", "Chicago", "Florence"
]

In [13]:
#|export
def generate_new_game():
    new_game_prompt = f"""
    System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
    User: To start the game, generate the following and return it as a json hash. 
    1. A random uuid, key: case_id
    2. A suspect name: suspect_name
    3. A source city from {famous_cities}, key: current_city 
    4. A famous artifact from that city, key: stolen_item
    5. A list of {MAX_HOPS + 1} unique cities from {famous_cities} with the current city as the first element, key: hops
    6. Set a field called next_hop to 1
    
    Make the suspect name and artifacts as campy as possible but use real city names. 

    Here is an example:
    {{
        case_id: <uuid>,
        suspect_name: <suspect_name>,
        current_city: <current_city>,
        stolen_item: <item name>,
        hops: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
            <city 5>,
            <city 6>,
        ],
        next_hop: 1
    }}
    
    Output only the json hash. Do not provide any other text output or additional formatting. 

    Game master:
    """

    return get_generation_llm().invoke(new_game_prompt).content
    

In [14]:
game_state = json.loads(generate_new_game())
game_state

{'case_id': 'd3f7c2f1-4588-4eac-a80f-2b1a4f54eac6',
 'suspect_name': 'Baron Von Schnitzel',
 'current_city': 'Paris',
 'stolen_item': 'Eiffel Tower miniature',
 'hops': ['Paris', 'Berlin', 'Moscow', 'Dubai', 'Sydney', 'Rio de Janeiro'],
 'next_hop': 1}

In [15]:
#|export
@retry(stop_max_delay=15000, stop_max_attempt_number=3)
def new_game():
    game_state = json.loads(generate_new_game())
    store_game_state(game_state)

    res_fields = ["case_id", "suspect_name", "current_city", "stolen_item"]
    res = {}

    for field in res_fields:
        res[field] = game_state[field]
        
    return res

In [16]:
new_game()

{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b',
 'suspect_name': 'Diamond Dave',
 'current_city': 'Paris',
 'stolen_item': 'Eiffel Tower miniature'}

In [17]:
!cat {GAME_STATE_FILE}

{"d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b": {"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Paris", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 1}}

In [18]:
case_id, _ = next(iter(get_game_states().items()))
case_id

'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'

In [19]:
#|export
@tool
def fetch_game_state(case_id: str) -> str:
    """ Fetch the game state given the case_id """
    return json.dumps(get_game_states()[case_id])

In [20]:
fetch_game_state.invoke({"case_id": case_id})

'{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Paris", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 1}'

In [21]:
#|export
@tool
def set_current_city(case_id: str, current_city: str):
    """ Sets the current city for the given case_id """
    game_state = get_game_state(case_id)
    game_state["current_city"] = current_city

    store_game_state(game_state)
    

In [22]:
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": "Mumbai"
    }
)

get_game_state(case_id)

{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b',
 'suspect_name': 'Diamond Dave',
 'current_city': 'Mumbai',
 'stolen_item': 'Eiffel Tower miniature',
 'hops': ['Paris',
  'London',
  'New York City',
  'Tokyo',
  'Sydney',
  'Rio de Janeiro'],
 'next_hop': 1}

In [23]:
#|export
@tool
def update_game_state(case_id, key, value):
    """ Updates the game state based on the values given """
    game_state = get_game_state(case_id)
    game_state[key] = value

    store_game_state(game_state)

In [24]:
update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": 3,
    }
)

get_game_state(case_id)

{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b',
 'suspect_name': 'Diamond Dave',
 'current_city': 'Mumbai',
 'stolen_item': 'Eiffel Tower miniature',
 'hops': ['Paris',
  'London',
  'New York City',
  'Tokyo',
  'Sydney',
  'Rio de Janeiro'],
 'next_hop': 3}

In [25]:
#|export
@tool
def generate_destinations(previous_city: str, next_city: str, current_city: str, exclude_list: str) -> str:
    """ Generates destinations for the player to travel to given the current city, previous city, 
    the next city, and a list of cities to exclude """
    prompt = f"""
        System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
        User: Generate a list of 4 cities. The list must include {previous_city}, {next_city} 
        and 2 other cities. It must not include {current_city} or any of the
        cities in {exclude_list}. Order the list so that {previous_city} and {next_city} are randomly
        interspersed in the list.
        
        Return this as a json list named destinations. 

        Here is an example of the json format:
        destinations: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]

        Do not include any additional text or additional formatting.
        Game Master:
        """

    return get_generation_llm().invoke(prompt).content

In [26]:
generate_destinations.invoke(
    {
        "previous_city": "London",
        "next_city": "Mumbai",
        "current_city": "Tokyo",
        "exclude_list": "Paris,Lima"
    }
)

'destinations: [\n            "Cairo",\n            "London",\n            "Sydney",\n            "Mumbai",\n        ]'

In [27]:
#|export
clue_locations = ["Tourism Desk", "Bank", "Embassy", "Restaurant", "Library"]

In [28]:
#|export
@tool
def generate_regular_clues(city: str) -> str:
    """ Generate regular clues given a location """
    prompt = f"""
        System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
        User: Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations from {clue_locations}. Pick them in random order.
        2. Generate a clue for each of the locations
        
        The clue for a bank should be the currency of the country. For example if the city is London, 
        the clue should say "The suspect changed their currency to Pounds"
        
        The clue for the embassy should be about the flag of the country. For example, if the city is 
        New York, the clue should say "The suspect enquired about visiting a country with a red, white, and blue flag"
        
        The clue for the restaurant should be a good from the country the city is in. For example, for Mumbai, the clue
        should say "The suspect was looking for fresh samosas"
        
        The clue for Tourism Desk should be something about a famous tourist attraction from the city. For example, for 
        Lima the clue should say "The suspect was interested in exploring Incan ruins"
        
        The clue for the library should be about a famous author from the country the city is in. For example, for London
        the clue could say "The suspect was interested in visiting the grave of Charles Dickens"
        
        Return it as a json hash with the following keys as a response:
        1. clues: a list of json hashes with keys location and clue
        
        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
        
        Do not generate any additional text or additional formatting.
       
        Now generate clues for {city}

        Game Master:
    """
    return get_generation_llm().invoke(prompt).content

In [29]:
json.loads(generate_regular_clues.invoke(
    {
        "city": "Bangalore" 
    }
))

{'clues': [{'location': 'Bank',
   'clue': 'The suspect exchanged their currency to Indian Rupees'},
  {'location': 'Embassy',
   'clue': 'The suspect asked about visiting a country with an orange, white, and green flag'},
  {'location': 'Restaurant',
   'clue': 'The suspect was craving spicy masala dosa'}]}

In [30]:
#|export
@tool
def generate_mistaken_clues() -> str:
    """ Generate mistaken clues """
    prompt = f"""
        System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
        User: Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of {clue_locations}. 
        2. Set the clue to "No one with the suspect's description was seen here"
        
        Return it as a json hash with the following keys as a response:
            a. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
                
        Do not generate any additional text or additional formatting.

        Now generate the clues.

        Game Master:
    """
    return get_generation_llm().invoke(prompt).content

In [31]:
json.loads(generate_mistaken_clues.invoke({}))

{'clues': [{'location': 'Restaurant',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Library',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Embassy',
   'clue': "No one with the suspect's description was seen here"}]}

In [32]:
#|export
@tool
def generate_arrest_clues(suspect_name: str) -> str:
    """ Generate arrest clues given the name of the suspect"""
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of {clue_locations}. 
        2. For two of them set the clue to "Watch your step. You are getting close"
        3. For the third, set the clue to "Congratulations! You have arrested the suspect, {suspect_name}
        
        Return it as a json hash with the following keys as a response:
            a. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
 
        Do not generate any additional text or additional formatting.

        Now generate the clues for {suspect_name} 
    """
    return get_generation_llm().invoke(prompt).content

In [33]:
json.loads(generate_arrest_clues.invoke(
    {
        "suspect_name": "Stealy McStealy"
    }
))

{'clues': [{'location': 'Library',
   'clue': 'Watch your step. You are getting close'},
  {'location': 'Tourism Desk',
   'clue': 'Watch your step. You are getting close'},
  {'location': 'Bank',
   'clue': 'Congratulations! You have arrested the suspect, Stealy McStealy'}]}

In [34]:
#|export
python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=PythonREPL().run,
)

In [35]:
#|export
tools = [
    fetch_game_state, 
    generate_destinations, 
    generate_regular_clues, 
    generate_arrest_clues, 
    generate_mistaken_clues,
    python_repl,
    update_game_state,   
]

In [36]:
#|export
def get_agent_executor():
    prompt = ChatPromptTemplate.from_messages(
        [
          ("system", "You are the game master for a detective game like 'where in the world is Carmen San Diego'. You will have to make a lot of decisions so look at the clues carefully."),
          ("placeholder", "{chat_history}"),
          ("human", "{input}"),
          ("placeholder", "{agent_scratchpad}"),
        ]
    )

    llm = get_agent_llm()
    # Construct the tool calling agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # Create an agent executor by passing in the agent and tools
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor

In [37]:
#|export
@retry(stop_max_delay=15000, stop_max_attempt_number=3)
def get_destinations(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    Do the following:
    1. Fetch the game state for case_id {case_id}
    2. Write python code passing in the game state to do the following:
        a. Extract the current_city from the game state obtained above
        b. If the value of next_hop is {MAX_HOPS + 1} return an empty list
        c. Get the city at the index next_hop from the hops list in the game state and save it as the next city
        d. If next_hop is 1 then set the previous city to "" otherwise get the city at the index next_hop -1 from 
            the hops list in the game state and save it as the previous city
        e. Add all other cities in the hops list to an exclude list
    
    3. Based on the output above, generate 4 destinations for the player to travel to including the previous city,
    current city, next city, and cities not in the exclude list. 
    4. Shuffle the list
    
    Return the output as a json hash with the following keys:
    1. city: The current city
    2. destinations: a list of the cities

    Here is an example of the output format:
    {{
        "city": <current city>
        "destinations": [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]
    }}
            
    Generate no additional text or additional formatting.
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return json.loads(ret['output'])

In [38]:
get_destinations(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Mumbai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}

Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl` with `game_state = {'suspect_name': 'Diamond Dave', 'current_city': 'Mumbai', 'stolen_item': 'Eiffel Tower miniature', 'hops': ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro'], 'next_hop': 3}

exclude_list = []

if game_state['next_hop'] == 6:
    []
else:
    next_city = game_state['hops'][game_state['next_hop']]
    if game_state['next_hop'] == 1:
        previous_city = ""
    else:
        previous_city = game_state['hops'][game_state['next_hop'] - 1]
    exclude_list = [city for city in game_state['hops']]
    exclude_list.remove(previous_city)
    exclude_list.remove(game_state['current_city'])
    exclude_list.remove(next_city)

{
    'current_city': game_state['current_city'],
    'previous_city': previous_city,
    'next_city': next_city,
    'exclude_list': exclude_list
}`


ValueError('list.remove(x): x not in list')
Invoking: `generate_destinations` with `{'current_city': 'Mumbai', 'exclude_list': 'Paris,London,New York City

{'city': 'Mumbai',
 'destinations': ['Dubai', 'New York City', 'Sydney', 'Tokyo']}

In [39]:
#|export
@retry(stop_max_delay=15000, stop_max_attempt_number=3)
def get_clues(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    1. Fetch the game state for case_id {case_id}.
    2. Write and execute python code passing in the game state to do the following to figure out what kind of 
       clue to generate:
        a. Extract next_hop from the game state
        b. If next_hop is equal to {MAX_HOPS + 1} only then print "arrest clues" as the type of clues
        c. Else If the current city is not in the hops list only then print "mistaken clues" as the type of clue
        d. Else print "regular clues" as the type of clue
    3. Based on the output of the code generate the right type of clue for the city at index next_hop in the hops list
    
    Return the output exactly as a hash encoded in json returned from the tool. Generate no additional text or
    additional formatting.
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return json.loads(ret['output'])

In [40]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Mumbai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b', 'suspect_name': 'Diamond Dave', 'current_city': 'Mumbai', 'stolen_item': 'Eiffel Tower miniature', 'hops': ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro'], 'next_hop': 3}

if game_state['next_hop'] == 6:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
    "clues": [
        {
            "location": "Library",
            "clue": 

{'clues': [{'location': 'Library',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Bank',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Embassy',
   'clue': "No one with the suspect's description was seen here"}]}

In [41]:
cur_city = get_game_state(case_id)["current_city"]
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": "Pune"
    }
)

get_clues(case_id)
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": cur_city
    }
)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Pune", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b', 'suspect_name': 'Diamond Dave', 'current_city': 'Pune', 'stolen_item': 'Eiffel Tower miniature', 'hops': ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro'], 'next_hop': 3}

if game_state['next_hop'] == 6:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
    "clues": [
        {
            "location": "Bank",
            "clue": "No one

In [42]:
nxt_hop = get_game_state(case_id)["next_hop"]
update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": MAX_HOPS + 1,
    }
)

get_clues(case_id)

update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": nxt_hop,
    }
)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Mumbai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 6}
Invoking: `python_repl` with `next_hop = 6
if next_hop == 6:
    clue_type = 'arrest clues'
else:
    current_city = 'Mumbai'
    hops_list = ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro']
    if current_city not in hops_list:
        clue_type = 'mistaken clues'
    else:
        clue_type = 'regular clues'
clue_type`



Invoking: `generate_arrest_clues` with `{'suspect_name': 'Diamond Dave'}`
responded: {"clue_type": "arrest clues"}

{
    "clues": [
        {
            "location": "Restaurant",
            "clue": "Watch your step. You are getting close"
        },
        {
      

In [43]:
#|export
@retry(stop_max_delay=15000, stop_max_attempt_number=3)
def travel(case_id, city):
    agent_executor = get_agent_executor()
    
    prompt = f"""
        1. Fetch the game state for case_id {case_id}
        2. Write python code passing in the game state to do the following:
            a. If {city} is not in the hops list, update current_city to {city} and print out the 
            new values of current_city and next_hop as a json hash
            b. Else if {city} is in the hops list and the index of {city} is <= next_hop then 
            set next_hop to the index + 1 and current_city to {city}. Print out the 
            new values of current_city and next_hop as a json hash
            c. Else if {city} is in hops list and the index of {city} is greater than next_hop 
            print the error "Error: You cannot travel to that city"

        If there is no error from the python code, 
        1. Update the current_city in the game state
        2. Update next_hop in the game state
        3. return the current city as json hash with a current_city key. 
        
        Else if there is an error from the python code, return the error "You cannot travel to that city" as a 
        json hash with the key "error".
        
        Here is an example of the output format when there is no error:
        {{
            "current_city": <city>
        }}

        Here is an example of the output format when there is an error:
        {{
            "error": "You cannot travel to that city"
        }}

        Generate no additional text or additional formatting.

    """

    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return json.loads(ret['output'])
    

In [44]:
travel(case_id, "New York")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Mumbai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}
Invoking: `python_repl` with `if 'New York' not in game_state['hops']: print(json.dumps({'current_city': 'New York', 'next_hop': 1}))`


NameError("name 'json' is not defined")
Invoking: `python_repl` with `elif 'New York' in game_state['hops'] and game_state['hops'].index('New York') <= game_state['next_hop']: print(json.dumps({'current_city': 'New York', 'next_hop': game_state['hops'].index('New York') + 1}) )`


SyntaxError('invalid syntax', ('<string>', 1, 1, "elif 'New York' in game_state['hops'] and game_state['hops'].index('New York') <= game_state['next_hop']: print(json.dumps({'current_city': 'New York',

{'error': 'You cannot travel to that city'}

In [45]:
get_game_state(case_id)

{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b',
 'suspect_name': 'Diamond Dave',
 'current_city': 'New York City',
 'stolen_item': 'Eiffel Tower miniature',
 'hops': ['Paris',
  'London',
  'New York City',
  'Tokyo',
  'Sydney',
  'Rio de Janeiro'],
 'next_hop': 3}

In [47]:
travel(case_id, "Chennai")
get_game_state(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Chennai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'current_city': 'Chennai', 'next_hop': 3, 'hops': ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro']}

if 'Chennai' not in game_state['hops']:
    game_state['current_city'] = 'Chennai'
    print({'current_city': game_state['current_city'], 'next_hop': game_state['next_hop']})
elif 'Chennai' in game_state['hops'] and game_state['hops'].index('Chennai') <= game_state['next_hop']:
    game_state['next_hop'] = game_state['hops'].index('Chennai') + 1
    game_state['current_city'] = 'Chennai'
    print({'current_city': game_state['current_city'], 'ne

{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b',
 'suspect_name': 'Diamond Dave',
 'current_city': 'Chennai',
 'stolen_item': 'Eiffel Tower miniature',
 'hops': ['Paris',
  'London',
  'New York City',
  'Tokyo',
  'Sydney',
  'Rio de Janeiro'],
 'next_hop': 3}

In [48]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b'}`


{"case_id": "d8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b", "suspect_name": "Diamond Dave", "current_city": "Chennai", "stolen_item": "Eiffel Tower miniature", "hops": ["Paris", "London", "New York City", "Tokyo", "Sydney", "Rio de Janeiro"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': 'd8ff95c7-5f71-4f02-a6a2-5e4b3d0a7f3b', 'suspect_name': 'Diamond Dave', 'current_city': 'Chennai', 'stolen_item': 'Eiffel Tower miniature', 'hops': ['Paris', 'London', 'New York City', 'Tokyo', 'Sydney', 'Rio de Janeiro'], 'next_hop': 3}

if game_state['next_hop'] == 6:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
    "clues": [
        {
            "location": "Library",
            "clue"

{'clues': [{'location': 'Library',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Restaurant',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'Tourism Desk',
   'clue': "No one with the suspect's description was seen here"}]}

In [49]:
from nbdev.export import nb_export

nb_export('./carmen.ipynb', './')

In [50]:
!ls

carmen_backend.py	carmen_client.ipynb	carmen.ipynb  server.py
carmen_backend_test.py	carmen_game_state.json	__pycache__
